### week 12

question 1

In [1]:
import pandas as pd

df = pd.read_csv('homework_12.1.csv')
df.head()

,Unnamed: 0,Y,Time,Group
0,0,-0.232900,-1.193204,0
1,1,2.848846,-1.607748,1
2,2,0.550209,-0.269793,0
3,3,2.198280,7.743730,0
4,4,4.111044,-4.244359,1


In [2]:
import numpy as np

df['post'] = df['Time'] > 0
did = (df[(df.Group==1)&(df.post)].Y.mean() - df[(df.Group==1)&(~df.post)].Y.mean()) - \
      (df[(df.Group==0)&(df.post)].Y.mean() - df[(df.Group==0)&(~df.post)].Y.mean())
did

np.float64(0.9440103783528997)

question 2

In [6]:
import statsmodels.formula.api as smf

df = pd.read_csv('homework_12.2.csv')
pre = df[df['Time'] < 0].copy()
pre['interaction'] = pre['Group'] * pre['Time']

model = smf.ols('Y ~ Group + Time + interaction', data=pre).fit()
model.tvalues['interaction']

np.float64(5.329379663924038)

question 3

In [8]:
# params from question
rho = 0.8
n = 10000
trials = 200

def generate_ar1_errors(n, rho):
    e = np.zeros(n)
    e[0] = np.random.normal()
    for t in range(1, n):
        e[t] = rho * e[t-1] + np.random.normal(scale=np.sqrt(1 - rho**2))
    return e

betas = []

for _ in range(trials):
    # X error term
    X_error = generate_ar1_errors(n, rho)
    
    # Y error term (independent new AR(1))
    Y_error = generate_ar1_errors(n, rho)
    
    X = X_error
    Y = 2 * X + Y_error
    
    X_mat = sm.add_constant(X)
    model = sm.OLS(Y, X_mat).fit()
    betas.append(model.params[1])

betas = np.array(betas)
print("Basic OLS SE (one run):", model.bse[1])
print("Simulation-based SE:", betas.std())


Basic OLS SE (one run): 0.009560999787033389
Simulation-based SE: 0.021770463523253333


question 4

In [7]:
import statsmodels.api as sm

np.random.seed(0)
X1 = np.random.normal(0,1,1000)
X2 = np.random.normal(0,1,1000)+X1
X3 = np.random.normal(0,1,1000)+2*X2

# regress X1 on X2 and X3
X = np.column_stack([X2,X3])
X = sm.add_constant(X)
res = sm.OLS(X1, X).fit()
r2 = res.rsquared
vif = 1/(1-r2)
vif

np.float64(1.9783742480901254)